In [42]:
previousFrame = None
count = 0
test = 0
temp_frame = None
rect = None
newThresh = None  

if __name__ == '__main__':

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('try6.avi',fourcc, 20.0, (640,480))
    #camera =cv2.VideoCapture("try3.avi")  # this for mobil connection cam
    camera =cv2.VideoCapture("http://192.168.0.104:4747/video")
    time.sleep(0.25)
    #global hists
    #hists = queue.Queue(10)
    #global prevSum
    #prevSum = 0
    global previousFrame
    #previousFrame = None
    while camera.isOpened():
        
        sensitivity_value = 20 
        ret, frame = camera.read()
        out.write(frame)
        #Check if the next camera read is not null
        if ret:
            frame, previousFrame, thresh, frameDiff, masks = trackMotion(frame,previousFrame, sensitivity_value)
            
            
            #get the contour for all the white areas in the frame
            _, contours, _ = cv2.findContours(masks,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
            
            #get the index of the largest contour
            index = getLargestCountour(contours)

            # if the index is -1 then there was no contours from the begining then move
            if index >=0 and index <= len(contours) -1:
                cnt = contours[index]

                #get the points of the parameter that shape the outer convex
                #false parameter means to retrieve with the hull the convexity  defects (the tie between two fingers)
                hull = cv2.convexHull(cnt,returnPoints = False)

                #get the ties
                defects = cv2.convexityDefects(cnt,hull)
                #draw the defects
                drawDefects(defects)
                imgsShow()
                
        else:
            print("Video Finished")
            close = False
            while not close:
                key1 = cv2.waitKey(3) & 0xFF
                if key1 == 27 or key1 == ord('q'):
                    close = True

            break
        key = cv2.waitKey(3) & 0xFF
        if key == 27 or key == ord('q'):
            print("Bye")
            break

    camera.release()
    cv2.destroyAllWindows()

81 0 49
113 32 81
81 0 58
113 32 90
13 93 20
24 125 52
13 109 23
24 141 55
13 91 51
27 123 83
10 3 71
42 35 103
77 3 33
91 29 65
77 3 33
91 23 65
77 6 90
91 26 122
80 3 90
91 23 122
80 3 90
91 23 122
80 3 90
91 23 122
77 6 90
91 23 122
80 6 90
91 26 122
80 3 90
91 20 122
80 3 90
91 23 122
80 6 90
91 23 122
80 6 90
91 26 122
77 6 90
91 23 122
80 6 90
91 29 122
80 6 90
91 23 122
80 6 90
91 26 122
77 6 90
91 23 122
80 6 90
91 23 122
80 6 90
91 23 122
80 6 90
91 26 122
74 6 90
97 38 122
80 6 91
91 23 123
80 6 91
91 26 123
74 6 91
100 35 123
80 6 94
91 29 126
77 6 93
91 23 125
7 3 93
36 35 125
13 96 19
24 128 51
10 87 10
42 119 42
13 84 16
24 116 48
7 3 48
39 35 80
4 3 58
36 35 90
7 3 61
39 35 93
7 3 68
39 35 100
10 36 55
36 68 87
10 36 58
42 68 90
10 3 61
42 35 93
10 3 58
42 35 90
7 3 58
39 35 90
10 3 58
42 35 90
10 3 55
42 35 87
10 75 58
24 107 90
10 3 55
33 35 87
13 6 55
33 38 87
10 3 55
27 35 87
10 3 55
27 35 87
10 78 58
24 110 90
10 3 58
27 35 90
13 3 58
27 35 90
10 3 58
27 35 90
10 77

In [16]:
#import imutils
import time
import math
import cv2
import numpy as np
import queue 
from matplotlib.pyplot import bar

In [17]:
#return only  the index of the largest contour
def getLargestCountour(contours):
    largestArea = 0
    index = 0
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>largestArea):
                largestArea=area
                index= i
    return index

In [24]:
#draw the defects
def drawDefects(defects):
    
    if defects is None:
        return
    #put inside each element the tie x,y and the color   true==> red   &  false ==> blue  & start of the finger and change in x and change in y
    vec = []

    #blue refer to reverse hand
    #count number of blue ties
    countDefectsBlue = 0

    #red refer to straight hand
    #count number of red ties
    countDefectsRed = 0
    countDefectsYellow = 0;
    
    try:
        for i in range(defects.shape[0]):

            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0]) #finger top
            end = tuple(cnt[e][0]) #another finger top
            tie = tuple(cnt[f][0]) # tie between them



            #if the distance between start line with end to the tie is less than 1000 this means that this not a tie (noise for example)
            if d < 1000: 
                continue



            cv2.line(frame,start,end,[0,255,0],2)  

            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2) # distance between finger top and another finger top
            b = math.sqrt((tie[0] - start[0])**2 + (tie[1] - start[1])**2) # distance between finger top and a tie
            c = math.sqrt((end[0] - tie[0])**2 + (end[1] - tie[1])**2) # distance between another finger top and a tie

            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 180/3.14
            dx = abs(start[0] - end[0])
            dy = abs(start[1] - end[1])

            #if the angle is less than 50 then it's not a tie
            if angle <= 70:
                if(dx > dy): #w - v - a -m
                    if abs(tie[1] - start[1])> dy and abs(tie[1] - end[1]) > dy:
                        # if the tie y axis is less than y axis for top finger or the other one then  the hand is reversed
                        if tie[1] < start[1] or tie[1] <end[1]:
                            countDefectsBlue+= 1 # increment the blue defects
                            vec.append((start,tie,1)) # push the tie into the array to print later

                        elif tie[1] > start[1] or tie[1] > end[1]:
                            countDefectsRed += 1 # increment the red defects
                            vec.append((start,tie,0)) # push the tie into the array to print later

            elif angle < 110 and angle >=90: #this to print the l  character                      
                    y0,y1 =start,end
                    if start[1] < end[1]:
                        y0 = start
                        y1 = end
                    else:
                        y0 = end
                        y1 = start
                    if abs(y0[0] - tie[0]) < abs(y1[0] - tie[0]):   #if the change in x axis between the finger and the tie is less than the shorter finger and the tie
                        countDefectsYellow += 1 # increment the red defects
                        vec.append((start,tie,2)) # push the tie into the array to print later



        #print the ties

        printTies(vec,countDefectsRed,countDefectsBlue,countDefectsYellow)  
        identifyChar(countDefectsRed,countDefectsBlue,countDefectsYellow)                   
    except Exception as e:
            print("hello")
            print(e)

In [19]:
'''
def getHistPeak(Hist,levelRange):
    PixelsCount = 0
    maxIndex = 0
    maxPixels = 0
    
    count = 0
    for i in range(256):
        count += Hist[i]
        if i+levelRange >255:
            break
        for j in range(i,i+levelRange):
            PixelsCount += Hist[j]
            
        if PixelsCount > maxPixels:
            maxIndex = i
            maxPixels = PixelsCount
        PixelsCount = 0
    startIndex , endIndex = getThreshold(Hist,maxIndex + int(levelRange/2),count)
    return startIndex , endIndex
'''

'\ndef getHistPeak(Hist,levelRange):\n    PixelsCount = 0\n    maxIndex = 0\n    maxPixels = 0\n    \n    count = 0\n    for i in range(256):\n        count += Hist[i]\n        if i+levelRange >255:\n            break\n        for j in range(i,i+levelRange):\n            PixelsCount += Hist[j]\n            \n        if PixelsCount > maxPixels:\n            maxIndex = i\n            maxPixels = PixelsCount\n        PixelsCount = 0\n    startIndex , endIndex = getThreshold(Hist,maxIndex + int(levelRange/2),count)\n    return startIndex , endIndex\n'

In [10]:
def getPixelsCount(hist):
    count = 0
    for i in range(256):
        count += hist[i]
    return count

In [11]:
def getThreshold(hist,maxIndex,count):
    #count = getPixelsCount(hist)
    threshold1 = maxIndex -1
    threshold2 = maxIndex +1
    p = 0
    while p < 0.65 and threshold2 - threshold1 < 30:
        p1,p2 = 0,0
        if threshold1 > 2:
            #p1 = hist[threshold1]/count 
            p1 = (hist[threshold1] + hist[threshold1-1]+ hist[threshold1-2])/count 
        if threshold2 < 254:
            #p2 = hist[threshold2]/count
            p2 = (hist[threshold2]+hist[threshold2+1]+hist[threshold2+2])/count
        if p1 > p2 and threshold1 > 2:
            p += (hist[threshold1] + hist[threshold1-1]+ hist[threshold1-2])/count 
            threshold1 -=3
        elif p1 <= p2 and threshold2 < 254:   
            p += (hist[threshold2]+hist[threshold2+1]+hist[threshold2+2])/count
            threshold2  += 3
        if p1 == p2 and p2 == 0:
            break
        #print(p)
    return threshold1 , threshold2 

In [39]:
def mask(frame,lowHue,lowSat,lowVal,highHue,highSat,highVal):
   #this is the limit threshold the lowest from the average value and the highest from the average value
    try:
        print(lowHue,lowSat,lowVal)
        print(highHue,highSat ,highVal)
    
        kernelDilation = np.ones((3,3),np.uint8) #kernel for dilation
        kernelErosion = np.ones((2,2),np.uint8)
        lower_blue = np.array([lowHue  ,lowSat - 255 ,lowVal])
        upper_blue = np.array([highHue ,highSat + 255,highVal ])
                
        mask0 = cv2.inRange(frame, lower_blue, upper_blue)
        
        lower_blue = np.array([lowHue  ,lowSat - 255 ,lowVal -255])
        upper_blue = np.array([highHue ,highSat + 255,highVal + 255 ])
        
        mask1 = cv2.inRange(frame, lower_blue, upper_blue)
        mask1 = cv2.erode(mask1,kernelErosion,iterations =1)
        mask1 = cv2.dilate(mask1,kernelDilation,iterations = 2)
        
        lower_blue = np.array([lowHue - 255  ,lowSat, lowVal -255])
        upper_blue = np.array([highHue + 255 ,highSat +255,highVal + 255 ])
        
        mask2 = cv2.inRange(frame, lower_blue, upper_blue)
        #mask2 = cv2.threshold (mask2, 250, 255, CV_THRESH_BINARY_INV);

        lower_blue = np.array([lowHue - 255  ,lowSat -255  ,lowVal - 50])
        upper_blue = np.array([highHue + 255 ,highSat + 255 ,highVal ])
        
        mask3 = cv2.inRange(frame, lower_blue, upper_blue)
        mask3 = cv2.dilate(mask3,kernelDilation,iterations = 2)
        #masks = mask1 * mask3
        masks = cv2.bitwise_and(mask1, mask3)
        masks = cv2.erode(masks,kernelDilation,iterations = 2)
        masks = cv2.dilate(masks,kernelDilation,iterations = 2)
        cv2.namedWindow('hue',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('hue', 800,600)
        cv2.imshow("hue", mask1)
        cv2.namedWindow('val',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('val', 800,600)
        cv2.imshow("val", mask3)
        #mask = mask1,mask3
        #mask = cv2.dilate(mask0,kernelDilation,iterations = 2)
    except Exception as e:
        print(e)
        return frame

    return masks

In [13]:
def trackMotion(frame,prev, sensitivity_value):
    
        masks = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Convert to grayscale and blur it for better frame difference
        # frame = cv2.bilateralFilter(frame, 7, 150, 150)
        frame = cv2.blur(frame, (9,9)) # blur the original frame "very important for the final image"


        previousFrame = prev
        global newThresh

        
        kernelErosion = np.ones((1,1),np.uint8) #kernel for erosion
        kernelDilation = np.ones((8,8),np.uint8) #kernel for dilation
        if previousFrame is None:
            previousFrame = frame.copy()
            prevHist =0
            #print('hello')
            return frame, previousFrame, frame, frame, masks

        frameDiff = cv2.absdiff(previousFrame, frame)
        #frameDiff = cv2.cvtColor(frameDiff, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.threshold(frameDiff, sensitivity_value, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
      
        hist = cv2.calcHist([thresh], [0], None, [256], [0,256])

        if hist[255] >= 5000 :
            previousFrame = frame.copy()
            newThresh = thresh # important
            erosion = cv2.erode(newThresh,kernelErosion,iterations =1)
            newThresh = cv2.dilate(newThresh, kernelDilation, iterations=4)
            newThresh = cv2.threshold(newThresh, 250, 255, cv2.THRESH_BINARY)[1]
    

        if newThresh is None:
                newThresh = thresh
        
        #find the contours from the remaining threshold image 
        _, cnts, _ = cv2.findContours(newThresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        #search for any movement
        frame, masks = searchForMovement(cnts, frame)    
        
        return frame, previousFrame, thresh, frameDiff, masks

In [22]:
def searchForMovement(contours, frame):
    #get the index of the largest contour
    index = getLargestCountour(contours)
    masks = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if index >=0 and index <= len(contours) -1:
        cnt = contours[index]
        M = cv2.moments(cnt) #get some properties from the biggest countour
        cx = int(M['m10']/M['m00']) #x axis centroid
        cy = int(M['m01']/M['m00']) #y axis centroid
        #cv2.circle(frame,(cx,cy),5,[255,0,0],-1) #draw the center of the countour mass
    
        hull = cv2.convexHull(cnt,returnPoints = False)
        minDist = 1000
        frame2 = frame.copy()
        for i in range(len(hull)):
            hullX = cnt[hull[i]][0][0][0]
            hullY = cnt[hull[i]][0][0][1]
            dist = math.sqrt( (hullX - cx)**2 + (hullY - cy)**2 )
            if dist < minDist:
                minDist = dist
            #cv2.circle(frame,(hullX,hullY),5,[255,255,0],-1)
        
        
        cv2.circle(frame,(cx,cy),int(minDist),[255,255,255],0)
        
        circlemask = np.zeros((frame2.shape[0],frame.shape[1]),dtype=np.uint8)
        cv2.circle(circlemask,(cx,cy),int(minDist - 2),(255,255,255),-1,8,0)
        #cv2.imwrite(argv[2],mask)
        '''
        out = frame.copy()
        out = cv2.bitwise_and(frame, out, mask=circlemask)
        cv2.imshow("out",out)
        '''
        #treshhold = 4
        levelRange = 4
        
        framehsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        
        #framehsv[:,:,0] = cv2.blur( framehsv[:,:,0], (6,6))
        circleHueHist = cv2.calcHist([framehsv], [0], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeakSat(circleHueHist,levelRange)   
        #index += int(levelRange / 2)
        lowHue =  treshold1
        highHue = treshold2
        
        '''
        img = framehsv[:,:,1] * circlemask
        ret2,th2 = cv2.threshold(img[img != 0] ,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #print(ret2)
        circleSatHist = cv2.calcHist([framehsv], [1], circlemask, [256], [0,256])
        treshold1,treshold2 = getThreshold(circleSatHist,int(ret2))     
        #index += int(levelRange / 2)
        lowSat = treshold1
        highSat = treshold2
        '''
        
        framehsv[:,:,1] = cv2.blur( framehsv[:,:,1], (9,9))
        circleSatHist = cv2.calcHist([framehsv], [1], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeakSat(circleSatHist,levelRange)  
        lowSat = treshold1
        highSat = treshold2
        
        #framehsv[:,:,1] = cv2.GaussianBlur(framehsv[:,:,1],(9,9),0)
        #framehsv[:,:,1] = cv2.equalizeHist(framehsv[:,:,1])
        #circleSatHist = cv2.calcHist([framehsv], [1], circlemask, [256], [0,256])
        #treshold1,treshold2 = getHistPeakSat(circleSatHist,10)
        #lowSat = treshold1
        #highSat = treshold2
        
        #equ = cv2.equalizeHist(framehsv[:,:,1])
        #ret2,th2 = cv2.threshold(framehsv[:,:,1],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #lowSat2, highSat2 = getThreshold(cv2.calcHist([equ],[0],None,[256],[0,256]),ret2)
        #lowSat2 = ret2 +50
        #highSat2 = ret2 + 70
        
        
        #blur = cv2.GaussianBlur(equ,(5,5),0)
        #ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #lowSat3 = ret3 - 10
        #highSat3 = ret3 + 10

        framehsv[:,:,2] = cv2.blur( framehsv[:,:,2], (9,9))
        circleValHist = cv2.calcHist([framehsv], [2], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeakSat(circleValHist,levelRange)  
        #index += int(levelRange / 2)
        lowVal = treshold1
        highVal = treshold2
        
        #masks1 = mask(framehsv,lowHue,lowSat,lowVal,highHue,highSat,highVal)
        masks = mask(framehsv,lowHue,lowSat,lowVal,highHue,highSat,highVal)
        #asks3 = mask(framehsv,lowHue,lowSat3,lowVal,highHue,highSat3,highVal)
        #cv2.imshow("masks1",masks1)
        #cv2.imshow("th2",th2)
        #cv2.imshow("masks1",masks1)
        #cv2.imshow("masks3",masks3)
        #v2.imshow("masks3",masks3)
        #cv2.imshow("mask1s",framehsv[:,:,1])
        #cv2.imshow("mask", masks)
        '''
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        
        cv2.drawContours(frame, [approx], -1, (0, 0, 255), 3)
        '''
        #cv2.drawContours(frame, hull, -1, (0, 0, 255), 3)
        '''
        defects = cv2.convexityDefects(cnt,hull)
        #hand = hsv[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
        #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html#contour-features
        if defects is not None:
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                cv2.line(frame,start,end,[0,255,0],2)
                rows,cols = frame.shape[:2]
                [vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2,0,0.01,0.01)
                lefty = int((-x*vy/vx) + y)
                righty = int(((cols-x)*vy/vx)+y)
                cv2.line(frame,(cols-1,righty),(0,lefty),(0,0,255),2)
        '''
    return frame, masks

In [4]:
def getHistPeakSat(hist,levelRange):
    maxIndex = 0
    c =0
    ci = cj = 0
    i=0
    j =255
    begin = True
    end = 255
    start = 0
    try:
     
        while end - start >20:
            while j > i:
                ci += hist[i]
                cj += hist[j]
                j -=1
                i +=1
            if begin == True:
                begin = False
                c = ci + cj + hist[128]
            if ci >= cj:
                i =start
                j =int((end+start)/2)
                end = j   
            if ci < cj:
                i = int((end+start)/2)
                j =end
                start = i
            ci = 0
            cj = 0
        maxIndex = int((start + end) / 2)
    except Exception as e: 
            print(e)
    startIndex , endIndex = getThreshold(hist,maxIndex,c)
    return startIndex , endIndex

In [5]:
#to print each tie according to the blue and red color and position
def printTies(vec,red,blue,yellow):
    if checkPoints(red,blue,yellow) == False:
        return
    for obj in vec:
        start = obj[0]
        tie = obj[1]
        color = obj[2]
        if color == 0: #red
              cv2.circle(frame,tie,5,[0,0,255],-1)
        elif color == 1: #blue
             cv2.circle(frame,tie,5,[255,0,0],-1)
        elif color == 2: #yellow
             cv2.circle(frame,tie,5,[0,255,255],-1)       
        cv2.line(frame,start,tie,[150,150,150],2)           
    vec.clear()   

In [6]:
def identifyChar(red,blue,yellow):
    if checkPoints(red,blue,yellow) == False:
        return
    if red == 1: #v
        showText("V")
    elif red == 2: #w
         showText("W")
    elif blue == 1: #a
         showText("A")
    elif blue == 2: #m
         showText("M")
    elif yellow == 1: #L
         showText("L")

In [7]:
#print the char on the screen
def showText(string):
            cv2.putText(frame,string, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 3, 3)

In [8]:
#check if the points are valid
def checkPoints(red,blue,yellow):
    if red > 0:
        if blue > 0 or yellow > 0 or red > 2: 
            return False
    if blue > 0:
        if red > 0 or yellow > 0 or blue > 2:
            return False
    if yellow > 0:
        if red > 0 or blue > 0 or yellow >1:
            return False
        
    return True

In [9]:
def imgsShow():
     #show images
        cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 800,600)
        cv2.imshow("frame", frame)
        cv2.namedWindow('mask',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('mask', 800,600)
        cv2.imshow("mask", masks)